<center>
<img src="https://supportvectors.ai/logo-poster-transparent.png" width="400px" style="opacity:0.7">
</center>

In [1]:
%run supportvectors-common.ipynb


<div style="color:#aaa;font-size:8pt">
<hr/>
&copy; SupportVectors. All rights reserved. <blockquote>This notebook is the intellectual property of SupportVectors, and part of its training material. 
Only the participants in SupportVectors workshops are allowed to study the notebooks for educational purposes currently, but is prohibited from copying or using it for any other purposes without written permission.

<b> These notebooks are chapters and sections from Asif Qamar's textbook that he is writing on Data Science. So we request you to not circulate the material to others.</b>
 </blockquote>
 <hr/>
</div>




# Vector Indices and Approximate Nearest Neighbor Search

## Table of Contents
1. [Introduction to Vector Indices](#introduction-to-vector-indices)
2. [Why Vector Indices?](#why-vector-indices)
3. [Approximate Nearest Neighbor (ANN) Search](#approximate-nearest-neighbor-ann-search)
4. [ANN Search Algorithms](#ann-search-algorithms)
5. [FAISS: Facebook AI Similarity Search](#faiss-facebook-ai-similarity-search)
6. [Vector Databases](#vector-databases)
7. [Qdrant: A Modern Vector Database](#qdrant-a-modern-vector-database)





## Introduction to Vector Indices

Vector indices are specialized data structures designed to efficiently store, organize, and retrieve high-dimensional vector representations of data. In the era of deep learning and AI, data is increasingly represented as dense vectors (embeddings) that capture semantic meaning in a continuous vector space.

### What are Embeddings?

Embeddings are dense vector representations that map discrete objects (text, images, audio, video) to points in a high-dimensional vector space. The key insight is that semantically similar objects are mapped to nearby points in this space.

**Mathematical Representation:**
Given an object $x$ and an embedding function $f$, we have:
$$f: \mathcal{X} \rightarrow \mathbb{R}^d$$
where $d$ is the dimensionality of the embedding space.

### Types of Embeddings

1. **Text Embeddings**: Generated by models like BERT, GPT, or sentence transformers
2. **Image Embeddings**: Produced by CNNs, Vision Transformers, or CLIP
3. **Audio Embeddings**: Created by models like Wav2Vec, Whisper
4. **Video Embeddings**: Generated by temporal models or frame aggregation
5. **Multimodal Embeddings**: Combining multiple modalities (e.g., CLIP for text-image pairs)





## Why Vector Indices?

As the dimensionality of vectors increases, traditional search methods become increasingly inefficient. 

**The Problem:**
- Linear search through $N$ vectors: $O(N)$ time complexity
- For high-dimensional spaces, distance calculations become expensive
- Traditional indexing methods (B-trees, hash tables) don't work well for similarity search

Traditional methods are optimized for:
- Exact matches
- Range queries
- Structured data

They are **not** optimized for:
- Similarity search
- High-dimensional vectors
- Approximate matching

Vector indices address these challenges by:
1. **Dimensionality Reduction**: Reducing the effective dimensionality
2. **Space Partitioning**: Dividing the vector space into manageable regions
3. **Approximation**: Trading exactness for speed
4. **Specialized Data Structures**: Optimized for vector operations





## Approximate Nearest Neighbor (ANN) Search

### Problem Definition

Given a query vector $q \in \mathbb{R}^d$ and a set of vectors $S = \{v_1, v_2, ..., v_n\}$, find the $k$ vectors in $S$ that are most similar to $q$.

**Mathematical Formulation:**
$$\text{ANN}(q, S, k) = \arg\min_{v \in S} \text{dist}(q, v)$$

where $\text{dist}$ is a distance metric (typically L2, cosine, or dot product). The function returns the $k$ vectors from $S$ that minimize the distance to $q$.

### Exact vs. Approximate Search

| Aspect | Exact Search | Approximate Search |
|--------|--------------|-------------------|
| **Accuracy** | 100% | 90-99% |
| **Speed** | Slow | Fast |
| **Memory** | High | Lower |
| **Scalability** | Poor | Excellent |




## ANN Search Algorithms



### 1. Locality-Sensitive Hashing (LSH)

LSH is a technique that hashes input items so that similar items map to the same "buckets" with high probability.

**Core Idea:**
- Use hash functions that preserve similarity
- Similar vectors are likely to hash to the same bucket
- Search only within the bucket of the query

**Pros:**
- Simple to implement
- Good theoretical guarantees
- Works well for high-dimensional data

**Cons:**
- Requires multiple hash tables for good recall
- Memory overhead
- Not optimal for very large datasets



### 2. Hierarchical Navigable Small World (HNSW)

HNSW constructs a hierarchical graph structure where each level contains a subset of the previous level, creating a "small world" network that enables efficient navigation through the vector space.

**Core Idea:**
- Build a multi-layer graph where each layer is a subset of the layer below
- Higher layers are sparser and enable fast "long-range" navigation
- Lower layers are denser and provide precise local search
- Navigate from top to bottom using greedy search, starting with coarse approximation and refining

**How HNSW Works:**

1. **Hierarchical Structure:**
   - **Top layer (Layer L)**: Very sparse, contains few points, enables fast global navigation
   - **Middle layers**: Moderate density, balance between speed and accuracy
   - **Bottom layer (Layer 0)**: Dense, contains all points, provides precise local search

2. **Graph Construction Process:**
   - Each point is assigned to layers probabilistically (higher layers with lower probability)
   - For each point, find its nearest neighbors in the current layer
   - Connect the point to these neighbors (up to M connections per layer)
   - Repeat for all layers the point belongs to

3. **Search Process:**
   - Start from the top layer with a **designated entry point** (not random - it's the highest-level point in the graph)
   - Find the **nearest neighbor to the query** in the current layer using local search
   - Use this nearest neighbor as the entry point for the layer below
   - Continue until reaching the bottom layer
   - Perform local search in the bottom layer to find final results



### 3. Inverted File Index (IVF)

IVF partitions the vector space into clusters and maintains an inverted index mapping clusters to vectors.

**Core Idea:**
1. Cluster vectors using k-means
2. Create inverted index: cluster_id → list of vectors
3. For queries, search only in relevant clusters

**Pros:**
- Simple and effective
- Good for large datasets
- Easy to tune

**Cons:**
- Performance depends on clustering quality
- Not optimal for very high-dimensional data



### 4. Product Quantization (PQ)

PQ reduces memory usage by quantizing vectors into smaller sub-vectors and encoding them.

**Core Idea:**
1. Split vector into $m$ sub-vectors
2. Quantize each sub-vector independently
3. Store only the quantization indices

**Pros:**
- Massive memory reduction
- Fast distance computation
- Good compression ratio

**Cons:**
- Loss of precision
- Complex implementation
- Not suitable for exact search




## FAISS: Facebook AI Similarity Search

### Introduction

FAISS (Facebook AI Similarity Search) is a library for efficient similarity search and clustering of dense vectors. It was developed by Facebook Research and has become a cornerstone in the ANN search ecosystem.

### Key Features

1. **Multiple Index Types**: Supports various ANN algorithms
2. **GPU Acceleration**: CUDA support for faster computation
3. **Memory Efficiency**: Optimized for large-scale datasets
4. **Flexibility**: Composable index structures

### Limitations of FAISS

1. **No Persistence**: Indexes must be rebuilt after restart
2. **No Metadata**: Can't store additional information with vectors
3. **No Updates**: Adding/removing vectors requires rebuilding
4. **No Filtering**: Can't filter results based on metadata
5. **Complex Configuration**: Many parameters to tune




## Vector Databases

### Why Vector Databases?

While FAISS is excellent for pure vector search, modern applications often need:

1. **Persistence**: Data survives restarts
2. **Metadata**: Store additional information with vectors
3. **Filtering**: Query by both vector similarity and metadata
4. **Updates**: Add/remove vectors without rebuilding
5. **Scalability**: Distributed storage and search
6. **Real-time**: Handle streaming data

### Popular Vector Databases

#### 1. Qdrant
- **Language**: Rust
- **Features**: High performance, rich filtering, real-time updates
- **Use Cases**: Production systems, real-time applications

#### 2. ChromaDB
- **Language**: Python
- **Features**: Easy to use, good for prototyping
- **Use Cases**: Research, development, small-scale applications

#### 3. Pinecone
- **Language**: Cloud service
- **Features**: Managed service, auto-scaling
- **Use Cases**: Production applications, enterprise

#### 4. Weaviate
- **Language**: Go
- **Features**: GraphQL interface, schema management
- **Use Cases**: Knowledge graphs, semantic search

#### 5. Milvus
- **Language**: C++
- **Features**: High performance, distributed
- **Use Cases**: Large-scale production systems

### Comparison Matrix

The following comparison is based on published benchmarks and performance studies. Note that performance characteristics can vary significantly based on workload, hardware, and configuration.

| Database | Language | Architecture | Key Strengths | Key Limitations | Best Use Cases |
|----------|----------|--------------|---------------|-----------------|----------------|
| **Qdrant** | Rust | Single-node, distributed | High performance, rich filtering, real-time updates | Limited ecosystem, newer product | Production systems, real-time applications |
| **ChromaDB** | Python | Embedded, client-server | Easy setup, good for prototyping | Limited scalability, performance overhead | Research, development, small-scale applications |
| **Pinecone** | Cloud | Managed service | Auto-scaling, zero maintenance | Vendor lock-in, cost at scale | Enterprise applications, managed solutions |
| **Weaviate** | Go | GraphQL-native | Schema management, knowledge graphs | Learning curve, resource intensive | Knowledge graphs, semantic search |
| **Milvus** | C++ | Distributed | High performance, billion-scale | Complex deployment, operational overhead | Large-scale production systems |

#### Performance Benchmarks

**Note**: The following benchmarks are from published studies and may not reflect current performance. Always conduct your own testing for your specific use case.

| Database | Query Latency (ms) | Throughput (QPS) | Memory Usage | Citation |
|----------|-------------------|------------------|--------------|----------|
| **Qdrant** | 1-5 | 10K-50K | Low | [Qdrant Benchmarks 2023](https://qdrant.tech/benchmarks/) |
| **ChromaDB** | 5-20 | 1K-5K | Medium | [ChromaDB Performance](https://docs.trychroma.com/performance) |
| **Pinecone** | 10-50 | 1K-10K | N/A (cloud) | [Pinecone Documentation](https://docs.pinecone.io/docs/performance) |
| **Weaviate** | 5-15 | 5K-20K | High | [Weaviate Benchmarks](https://weaviate.io/developers/weaviate/benchmarks) |
| **Milvus** | 1-10 | 50K-200K | Low | [Milvus Benchmarks](https://milvus.io/docs/benchmark.md) |

#### Feature Comparison

| Feature | Qdrant | ChromaDB | Pinecone | Weaviate | Milvus |
|---------|--------|----------|----------|----------|--------|
| **Open Source** | ✅ | ✅ | ❌ | ✅ | ✅ |
| **Cloud Managed** | ❌ | ❌ | ✅ | ✅ | ✅ |
| **Real-time Updates** | ✅ | ✅ | ✅ | ✅ | ✅ |
| **Rich Filtering** | ✅ | ⚠️ | ✅ | ✅ | ✅ |
| **Horizontal Scaling** | ✅ | ❌ | ✅ | ✅ | ✅ |
| **GPU Support** | ✅ | ❌ | ✅ | ⚠️ | ✅ |
| **GraphQL API** | ❌ | ❌ | ❌ | ✅ | ❌ |
| **REST API** | ✅ | ✅ | ✅ | ✅ | ✅ |

#### Selection Guidelines

**Choose Qdrant if:**
- You need high performance with rich filtering
- You want open-source with production readiness
- You have complex query requirements

**Choose ChromaDB if:**
- You're prototyping or doing research
- You need quick setup and easy integration
- You have small to medium datasets

**Choose Pinecone if:**
- You want managed service with auto-scaling
- You need enterprise support
- You prefer cloud-native solutions

**Choose Weaviate if:**
- You need knowledge graph capabilities
- You want GraphQL-native interface
- You're building semantic search applications

**Choose Milvus if:**
- You need billion-scale vector search
- You have the infrastructure for complex deployments
- You require maximum performance

*Sources: [Vector Database Benchmark 2023](https://zilliz.com/comparison), [Qdrant Benchmarks](https://qdrant.tech/benchmarks/), [Milvus Performance Guide](https://milvus.io/docs/performance_faq.md)*




## Qdrant: A Modern Vector Database

### Introduction

Qdrant is a high-performance vector database written in Rust, designed for production use cases requiring real-time vector search with rich filtering capabilities.

### Key Features

1. **High Performance**: Rust implementation for speed and memory safety
2. **Rich Filtering**: Complex boolean expressions on metadata
3. **Real-time Updates**: Add/remove vectors without downtime
4. **Horizontal Scaling**: Distributed architecture
5. **Multiple Distance Metrics**: L2, cosine, dot product
6. **Payload Management**: Flexible metadata storage

### Architecture

#### Core Components
1. **Storage Engine**: RocksDB for persistence
2. **Vector Index**: HNSW or IVF for ANN search
3. **Payload Index**: B-tree for metadata filtering
4. **API Layer**: REST and gRPC interfaces
